In [1]:
import time
import numpy as np
import cv2
import pandas as pd
import os


def erodeimg(image):
    kernel = np.ones((4, 4), np.uint8)
    erode = cv2.erode(image, kernel, iterations=1)
    return erode


def dilateimg(image):
    kernel = np.ones((3, 3), np.uint8)
    dilate = cv2.dilate(image, kernel, iterations=1)
    return dilate


def pix_count(image):
    real = image[image > 5]
    pixels = len(real)
    return pixels


def MotionParameters(image):
    eroded = erodeimg(image)
    dilated = dilateimg(eroded)

    mask = cv2.inRange(dilated, 5, 200)
    kernel = np.ones((10, 10), np.uint8)
    close = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    _, contours, hierarchy = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]

    if (len(contours) >= 1) & np.any(areas):
        max_i = np.argmax(areas)
        contour_ = contours[max_i]
        contour_h = cv2.convexHull(contour_)

        M = cv2.moments(contour_)
        hM = cv2.moments(contour_h)
        centroid_ = (int(M["m10"] / M["m00"])), (int(M["m01"] / M["m00"]))
        centroid_h = (int(hM["m10"] / hM["m00"])), (int(hM["m01"] / hM["m00"]))
    else:
        contour_ = ()
        contour_h = ()
        centroid_ = (np.nan, np.nan)
        centroid_h = (np.nan, np.nan)
    return contour_, contour_h, centroid_, centroid_h


def VideoList(path):
    video_list = [(os.path.join(root, file), file[0:-4])
                  for root, dirs, files in os.walk(path)
                  for file in files
                  if file.endswith('.mp4')]
    return video_list


def VideoPull(back_frame, current_frame, file_name, path):
    video = [(os.path.join(root, file), file[0:-4])
             for root, dirs, files in os.walk(path)
             for file in files
             if file.endswith(file_name + '.mp4')]
    cam = cv2.VideoCapture(video[0][0])
    cam.set(1, back_frame)
    ret, f = cam.read(1)
    background = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    cam.set(1, current_frame)
    ret, c = cam.read(1)
    current = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
    difference = cv2.subtract(current, background)

    return background, current, difference


path = "C:\\Users\\tanvid2\\MahadAnalysis\\VideosAnalyzed"
output_path = 'C:\\Users\\tanvid2\\MahadAnalysis\\Data'

video_list = VideoList(path)

for video in video_list:
    t0 = time.time()
    cam_ = video[0]
    name = video[1]

    cam = cv2.VideoCapture(cam_)
    back_frame = 0
    cam.set(1, back_frame)
    ret, f = cam.read(1)

    if not ret:
        with open(os.path.join(output_path, 'Logfile.txt'), 'a') as file1:
            file1.write(name + '\t' + ' did not load on' + '\t' + '\t' + str(time.asctime()) + '\n')

        print(name + ' not loaded')
    else:
        with open(os.path.join(output_path, 'Logfile.txt'), 'a') as file1:
            file1.write(name + '\t' + ' loaded successfully on' + '\t' + '\t' + str(time.asctime()) + '\n')
        print(name + 'loaded successfully')

        total_frames = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))
        # start_frame = 31000
        # end_frame = 31015
        # frameNum = frames[start_frame:end_frame]
        centroids_x = []
        centroids_y = []
        contours_hull = []
        centroids_hull_x = []
        centroids_hull_y = []
        pixels_per_frame = []

        for frame in range(0, total_frames):
            back, cur, diff = VideoPull(0, frame, name, path)
            if not np.any(cur):
                with open(os.path.join(output_path, 'Logfile.txt'), 'a') as file1:
                    file1.write(
                        name + '\t' + ' frame lost:' + '\t' + str(frame) + '\t' + 'on ' + str(time.asctime()) + '\n')
                print(name + ' frame num ' + str(frame) + 'lost')

            _, _, centroid, centroid_hull = MotionParameters(diff)

            pix = pix_count(diff)
            pixels_per_frame.append(pix)
            centroids_x.append(centroid[0])
            centroids_y.append(centroid[1])
            centroids_hull_x.append(centroid_hull[0])
            centroids_hull_y.append(centroid_hull[1])

    t1 = time.time()
    dt = t1 - t0
    with open(os.path.join(output_path, 'Logfile.txt'), 'a') as file1:
        file1.write(name + '\t' + ' took ' + '\t' + str(dt) + '\t' + 'seconds to complete' + '\n')
    print(name + ' done. It took ' + str(dt) + 'seconds')

    df = pd.DataFrame({'pixel': pixels_per_frame,
                       'Cent_x': centroids_x,
                       'Cent_y': centroids_y,
                       'Cent_hx': centroids_hull_x,
                       'Cent_hy': centroids_hull_y})
    df.to_csv(output_path + '/' + name + '_rawData.csv')